In [40]:
from pandas import read_csv, DataFrame, concat
from datetime import datetime
import matplotlib.pyplot as pyplot
from sklearn.preprocessing import MinMaxScaler
import numpy as np 

def parse(x):
    return datetime.strptime(x, '%m/%d/%Y')

# Data Loading
data_set = read_csv('march2020_2.csv', header=0, date_parser = parse)
data_set = data_set.iloc[:data_set['totalSoybeanMealSupply'].last_valid_index(),:] # Truncate dataset to those where every data is available 
target_col = ['closePrice','totalSoybeanMealSupply','totalSoybeanMealDemand']
data_set = data_set.loc[:,target_col]

In [41]:
values = data_set.values
values = values.astype('float32')
'''
pyplot.figure()
for i,col in enumerate(interest):
    ax = pyplot.subplot(len(interest), 1, i+1)
    pyplot.plot([datetime.strptime(x,'%m/%d/%Y').date() for x in values[:,0]],values[:, col])
    pyplot.title(data_set.columns[col], y=0.7, loc='right')
    ax.locator_params(axis='y', nbins=4)
    ax.locator_params(axis='x', nbins=6)
pyplot.show()
'''

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(values)




In [48]:
# Data Analysis
def series_to_supervised(data, y_col=0):
    agg = [] 
    for i in range(1, len(data)):
        row = np.concatenate((data[i-1],[data[i][y_col]]))
        agg += [row]
    return agg

data = np.asarray(series_to_supervised(scaled))
train_count = int(len(data) * 0.2)
train_data = data[:train_count]
test_data = data[train_count:]
train_x, train_y = train_data[:,:-1], train_data[:,-1]
test_x, test_y = test_data[:,:-1], test_data[:,-1]

# Reshape input to have the form (sample, timestep, features)
train_x = train_x.reshape((train_x.shape[0],1,train_x.shape[1]))
train_x = train_x.reshape((train_x.shape[0],1,train_x.shape[1]))


array([[0.7786379 , 0.5601649 , 0.6166668 ],
       [0.8188853 , 0.5601649 , 0.6166668 ],
       [0.8065014 , 0.5601649 , 0.6166668 ],
       [0.89009285, 0.5601649 , 0.6166668 ],
       [0.9024768 , 0.5601649 , 0.6166668 ],
       [0.8993807 , 0.5601649 , 0.6166668 ],
       [0.9303403 , 0.5601649 , 0.6166668 ],
       [0.9148607 , 0.5601649 , 0.6166668 ],
       [0.92260027, 0.5601649 , 0.6166668 ],
       [0.92260027, 0.5601649 , 0.6166668 ],
       [0.91331244, 0.5601649 , 0.6166668 ],
       [0.8978329 , 0.5601649 , 0.6166668 ],
       [0.9396286 , 0.63983536, 0.49806786],
       [0.9411764 , 0.63983536, 0.49806786],
       [0.9736843 , 0.63983536, 0.49806786],
       [0.95046425, 0.63983536, 0.49806786],
       [0.89473677, 0.63983536, 0.49806786],
       [0.88699675, 0.63983536, 0.49806786],
       [0.8622289 , 0.63983536, 0.49806786],
       [0.83436537, 0.63983536, 0.49806786],
       [0.85294104, 0.63983536, 0.49806786],
       [0.8126936 , 0.63983536, 0.49806786],
       [0.